In [31]:
import os

home = os.path.expanduser("~")

db_name=  'colab_main_dataset' #'test_new_creators_4'
input_dir = os.path.join(home, 'video_resources', 'image_datasets')
output_dir = os.path.join(home, 'video_resources', 'sql_database')
database_names=['enfsi','lfw','forenface','scface'] # 'lfw', 'scface',  'enfsi', 'enfsi2015', 'xqlfw', 'utkface','chokepoint','forenface'
detector_names=['mtcnn'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["QMagFace"] # , "Dlib", "ArcFace",
attributes_to_update=[]  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',

In [32]:
from sql_face.sqldb import SQLDataBase

database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

In [33]:

import pandas as pd
from tqdm import tqdm
from sql_face.tables import EnfsiPair, EnfsiPair2015, QualityImage, FaceImage, CroppedImage,Image, Quality

session = database.session

sql_1 = (session.query(QualityImage,Image.image_id)    
    .join(FaceImage)
    .join(CroppedImage)
    .join(Image)
    .filter(QualityImage.quality != None, Image.image_quality == None,
        #CroppedImage.image_id == Image.image_id,
        CroppedImage.detector_id== 2,
        #FaceImage.croppedImage_id==CroppedImage.croppedImage_id,
        FaceImage.embeddingModel_id==1,
        #QualityImage.faceImage_id==FaceImage.faceImage_id,
        QualityImage.qualityModel_id==1)
    .limit(200000).all())


print(len(sql_1))


58983


In [34]:
n1 = 100
n = n1
for Q,id in tqdm(sql_1):
    sql2 = (session.query(Image).filter(Image.image_id==id).one_or_none())
    if sql2:
        if Q.quality > 0.8:
            sql2.image_quality = Quality.VERY_HIGH
        elif Q.quality > 0.6:
            sql2.image_quality = Quality.HIGH
        elif Q.quality > 0.4:
            sql2.image_quality = Quality.MEDIUM
        elif Q.quality > 0.2:
            sql2.image_quality = Quality.LOW
        elif Q.quality > 0.0:
            sql2.image_quality = Quality.VERY_LOW
        
        if n:
            n -= 1
        else:
            session.commit()
            n = n1


session.commit()


100%|██████████| 58983/58983 [14:09<00:00, 69.41it/s]
